In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import re
from contracted_basis import CAO_basis, shell_pair
from basic_operations import operation as op
from fast_multipole_method import fmm_box

In [2]:
#testcase6: artifical system C2H2


ERIs = []
fr = open("C2H2.out", 'r')
while True:
    line = fr.readline()
    if line == "":
        break
    
    if line.find("Two-Electron Integrals") != -1:
        while True:
            line = fr.readline()
            if line == "\n":
                break
            ERIs.append(float(re.split("[,()|]|\s ", line)[7]))
            
fr.close()

In [3]:
len(ERIs)

20736

In [4]:
12 ** 4

20736

In [5]:
(1e-10 / np.sqrt(max(ERIs)))**2

2.8233434032581385e-21

In [6]:
for i in range(0, 12):
    for j in range(0, 12):
        if ERIs[i*(12**3) + j*(12*12) + i*12 + j] > 2.82e-21:
            print([i,j])


[0, 0]
[0, 1]
[0, 2]
[0, 3]
[0, 4]
[0, 5]
[1, 0]
[1, 1]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[2, 0]
[2, 1]
[2, 2]
[2, 3]
[2, 4]
[2, 5]
[3, 0]
[3, 1]
[3, 2]
[3, 3]
[3, 4]
[3, 5]
[4, 0]
[4, 1]
[4, 2]
[4, 3]
[4, 4]
[4, 5]
[5, 0]
[5, 1]
[5, 2]
[5, 3]
[5, 4]
[5, 5]
[6, 6]
[6, 7]
[6, 8]
[6, 9]
[6, 10]
[6, 11]
[7, 6]
[7, 7]
[7, 8]
[7, 9]
[7, 10]
[7, 11]
[8, 6]
[8, 7]
[8, 8]
[8, 9]
[8, 10]
[8, 11]
[9, 6]
[9, 7]
[9, 8]
[9, 9]
[9, 10]
[9, 11]
[10, 6]
[10, 7]
[10, 8]
[10, 9]
[10, 10]
[10, 11]
[11, 6]
[11, 7]
[11, 8]
[11, 9]
[11, 10]
[11, 11]


In [7]:
C1_x = np.array([1.09294e+01, -5.46669e-01, 1.70075e-05])
H1_x = np.array([1.10290e+01, -1.74631e+00, -1.66212e+00])
C2_x = np.array([-1.09294e+01, 5.46669e-01, 4.72432e-05])
H2_x = np.array([-1.10289e+01, 1.74633e+00, 1.66218e+00])
atom_x_scale = np.vstack((C1_x, H1_x, C2_x, H2_x))

In [8]:
atom_x

NameError: name 'atom_x' is not defined

In [9]:
[atom_x_scale, scale_factor] = op.cartesian_scaling_to_unit_range(atom_x)
atom_x_scale

NameError: name 'atom_x' is not defined

In [10]:
basis = np.ndarray(12, dtype=CAO_basis)
basis[0] = CAO_basis(atom_x_scale[0], "C", 1, np.array([0, 0, 0]))
basis[1] = CAO_basis(atom_x_scale[0], "C", 2, np.array([0, 0, 0]))
basis[2] = CAO_basis(atom_x_scale[0], "C", 2, np.array([1, 0, 0]))
basis[3] = CAO_basis(atom_x_scale[0], "C", 2, np.array([0, 1, 0]))
basis[4] = CAO_basis(atom_x_scale[0], "C", 2, np.array([0, 0, 1]))
basis[5] = CAO_basis(atom_x_scale[1], "H", 1, np.array([0, 0, 0]))
basis[6] = CAO_basis(atom_x_scale[2], "C", 1, np.array([0, 0, 0]))
basis[7] = CAO_basis(atom_x_scale[2], "C", 2, np.array([0, 0, 0]))
basis[8] = CAO_basis(atom_x_scale[2], "C", 2, np.array([1, 0, 0]))
basis[9] = CAO_basis(atom_x_scale[2], "C", 2, np.array([0, 1, 0]))
basis[10] = CAO_basis(atom_x_scale[2], "C", 2, np.array([0, 0, 1]))
basis[11] = CAO_basis(atom_x_scale[3], "H", 1, np.array([0, 0, 0]))

In [16]:
box1 = fmm_box(np.array([9, 0, 0]))
box2 = fmm_box(np.array([-9, 0, 0]))

In [33]:
sp_in_box1 = np.ndarray(36, dtype=shell_pair)
for i in range(0,6):
    for j in range(0,6):
        sp_in_box1[i*6+j] = shell_pair(basis[i], basis[j])

In [39]:
sp_in_box2 = np.ndarray(36, dtype=shell_pair)
for k in range(6,12):
    for l in range(6,12):
        sp_in_box2[(k-6)*6+(l-6)] = shell_pair(basis[k], basis[l])

In [42]:
ERIs_by_fmm = np.ndarray(36*36)
ERIs_far_component = np.ndarray(36*36)
p = 5
for i in range(0,6):
    for j in range(0,6):
        for k in range(6, 12):
            for l in range(6, 12):
                print([i,j,k,l])
                ERIs_far_component[i*(6**3)+j*(6**2)+(k-6)*6+(l-6)] = ERIs[i*(12**3)+j*(12**2)+k*12+l]
                box1 = fmm_box(np.array([0.95, 0.5, 0.5]))
                box2 = fmm_box(np.array([0.05, 0.5, 0.5]))
                sp_in_box1[i*6+j].M_expansion_to_box(box1, p)
                sp_in_box2[(k-6)*6+(l-6)].M_expansion_to_box(box2, p)
                box1.box_interaction(box2)
                ERIs_by_fmm[i*(6**3)+j*(6**2)+(k-6)*6+(l-6)] = sp_in_box1[i*6+j].Mlm.product(box1.Llm).sum().real
                
                

[0, 0, 6, 6]


IndexError: invalid index to scalar variable.

In [30]:
len(ERIs_far_component)

1296

In [ ]:
from fmm_source import gs_q_dist

In [ ]:
c_k = np.exp( - sum((basis[i].x - basis[j].x)**2)*a.a_k )
c_k = c_k.reshape(9) * a.d.reshape(9)
c_a = a.a_p.reshape(9) 
d_k = np.exp( - sum((basis[k].x - basis[l].x)**2)*b.a_k )
d_k = d_k.reshape(9) * b.d.reshape(9)
d_a = b.a_p.reshape(9) 

In [ ]:
c = np.ndarray(9, dtype=gs_q_dist)
d = np.ndarray(9, dtype=gs_q_dist)
for m in range(0, 9):
    c[m] = gs_q_dist(a.X_p[m], c_a[m], c_k[m])
    d[m] = gs_q_dist(b.X_p[m], d_a[m], d_k[m])

In [ ]:
direct_sum = 0
for i in range(0, 9):
    for j in range(0, 9):
        direct_sum += c[i].near_field_interaction(d[j], scale_factor[1])

In [ ]:
direct_sum 

In [ ]:
scale_factor[1]

In [ ]:
b.d.reshape(9)

In [ ]:
np.exp( - sum((basis[k].x - basis[l].x)**2)*b.a_k)

In [ ]:
b.a_k

In [ ]:
scale_factor = [0,1]

In [ ]:
i=0; j=0; k=8; l=10
box1 = fmm_box(np.array([9, 0, 0]))
box2 = fmm_box(np.array([-9, 0, 0]))
p = 20

In [ ]:
a = shell_pair(basis[i], basis[j])
b = shell_pair(basis[k], basis[l])

In [ ]:
a.M_expansion_to_box(box1, p)

In [ ]:
b.M_expansion_to_box(box2, p)

In [ ]:
box1.box_interaction(box2)
box2.box_interaction(box1)

In [ ]:
a.Mlm.product(box1.Llm).sum().real / scale_factor[1]

In [ ]:
ERIs[i*(12**3) + j*(12*12) + k*12 + l] 

In [ ]:
b.Mlm.V_matrix

In [ ]:
a.Mlm.V_matrix

In [ ]:
b.X_p

In [ ]:
i=0; j=0; k=6; l=11
a = shell_pair(basis[i], basis[j])
b = shell_pair(basis[k], basis[l])
box1 = fmm_box(np.array([0.95, 0.5, 0.5]))
box2 = fmm_box(np.array([0.05, 0.5, 0.5]))
p = 2
a.M_expansion_to_box(box1, p)
b.M_expansion_to_box(box2, p)
box1.box_interaction(box2)
box2.box_interaction(box1)

In [ ]:
a.Mlm.product(box1.Llm).sum().real / scale_factor[1]

In [ ]:
b.Mlm.V_matrix 

In [ ]:
a.Mlm.V_matrix

In [ ]:
b.X_p